In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import(precision_score, recall_score, f1_score, accuracy_score, mean_squared_error, mean_absolute_error)
from sklearn.preprocessing import Normalizer
from sklearn import model_selection as ms
from sklearn import preprocessing as pp
from tensorflow.keras.layers import BatchNormalization
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv1D,MaxPooling1D
from keras.layers import Dense, Dropout,Flatten, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import h5py

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
model_path='/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/Deep CNN Model/results/cnn_deep_Train+_Test+.h5'

# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_acc', 
        patience=10,
        mode='max',
        verbose=1),
    ModelCheckpoint(model_path,
        monitor='val_acc', 
        save_best_only=True, 
        mode='max',
        verbose=0)
]

In [4]:
# get information from NSL-KDD Dataset

class dataset:
    mawi_train_2labels = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/KDD Dataset Preprocessing/kdd_train_2labels.pkl")
    mawi_test_2labels = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/KDD Dataset Preprocessing/kdd_test_2labels.pkl")

class preprocess:
    output_columns_2labels = ['is_Normal','is_Attack']

    x_input = dataset.mawi_train_2labels.drop(output_columns_2labels, axis = 1)
    y_output = dataset.mawi_train_2labels.loc[:,output_columns_2labels]

    x_test_input = dataset.mawi_test_2labels.drop(output_columns_2labels, axis = 1)
    y_test = dataset.mawi_test_2labels.loc[:,output_columns_2labels]
    
    ss = pp.StandardScaler()
    
    x_train = ss.fit_transform(x_input)
    x_test = ss.fit_transform(x_test_input)
    
    y_train = y_output.values
    y_test = y_test.values
        
    print('x_train shape: {}'.format(x_train.shape))
    print('y_train shape: {}'.format(y_train.shape))
    print('x_test shape: {}'.format(x_test.shape))
    print('y_test shape: {}'.format(y_test.shape))

x_train shape: (125973, 122)
y_train shape: (125973, 2)
x_test shape: (22544, 122)
y_test shape: (22544, 2)


In [5]:
#reshape data
X_train = np.reshape(preprocess.x_train, (preprocess.x_train.shape[0], preprocess.x_train.shape[1], 1))
X_test = np.reshape(preprocess.x_test, (preprocess.x_test.shape[0], preprocess.x_test.shape[1], 1))

print('train shape after reshape: {}'.format(X_train.shape))
print('test shape after reshape: {}'.format(X_test.shape))

train shape after reshape: (125973, 122, 1)
test shape after reshape: (22544, 122, 1)


In [6]:
#size of parameters
batch_size = 64
num_classes = 2
epochs = 10
filter_size=3
droprate=0.50

In [7]:
#Start Neural Network
model = Sequential()

#convolution 1st layer
model.add(Conv1D(64, kernel_size=(filter_size), padding="same",
                 activation='relu',
                 input_shape=(122, 1)))
model.add(BatchNormalization())
model.add(Dropout(droprate))

#convolution 2nd layer
model.add(Conv1D(128, kernel_size=(filter_size), padding="same",
                 activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(droprate))
model.add(MaxPooling1D(strides=2))

#convolution 3rd layer
model.add(Conv1D(256, kernel_size=(filter_size), padding="same",
                 activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(droprate))
model.add(MaxPooling1D(strides=2))

#Flatten
model.add(Flatten())

#FCN Layer
model.add(Dense(64,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#FCN Layer
model.add(Dense(32,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#FCN Layer
model.add(Dense(16,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 122, 64)           256       
                                                                 
 batch_normalization (BatchN  (None, 122, 64)          256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 122, 64)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 122, 128)          24704     
                                                                 
 batch_normalization_1 (Batc  (None, 122, 128)         512       
 hNormalization)                                                 
                                                  

In [8]:
history = model.fit(X_train, preprocess.y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, preprocess.y_test),shuffle=True,callbacks=callbacks)

score = model.evaluate(X_test, preprocess.y_test, verbose=0)

#print loss and accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 125973 samples, validate on 22544 samples
Epoch 1/10
125973/125973 [==============================] - ETA: 0s - loss: 0.1686 - acc: 0.9549

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


125973/125973 [==============================] - 321s 3ms/sample - loss: 0.1686 - acc: 0.9549 - val_loss: 0.9208 - val_acc: 0.7651
Epoch 2/10
125973/125973 [==============================] - 324s 3ms/sample - loss: 0.0670 - acc: 0.9805 - val_loss: 1.0384 - val_acc: 0.7939
Epoch 3/10
125973/125973 [==============================] - 322s 3ms/sample - loss: 0.0579 - acc: 0.9817 - val_loss: 1.2880 - val_acc: 0.7717
Epoch 4/10
125973/125973 [==============================] - 323s 3ms/sample - loss: 0.0501 - acc: 0.9843 - val_loss: 1.3910 - val_acc: 0.7695
Epoch 5/10
125973/125973 [==============================] - 323s 3ms/sample - loss: 0.0471 - acc: 0.9846 - val_loss: 1.4224 - val_acc: 0.7800
Epoch 6/10
125973/125973 [==============================] - 323s 3ms/sample - loss: 0.0433 - acc: 0.9859 - val_loss: 1.4086 - val_acc: 0.7798
Epoch 7/10
125973/125973 [==============================] - 324s 3ms/sample - loss: 0.0416 - acc: 0.9866 - val_loss: 1.5465 - val_acc: 0.7778
Epoch 8/10
125973

In [9]:
#Confusion Matrix
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

predict_proba = model.predict(X_test)
predict_classes = np.argmax(predict_proba, axis=1)

target_name=["is_Attack", "is_Normal"]

print(classification_report(np.argmax(preprocess.y_test, axis=1), y_pred, target_names=target_name, digits=4))
print(confusion_matrix(np.argmax(preprocess.y_test, axis=1), y_pred))

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


              precision    recall  f1-score   support

   is_Attack     0.7206    0.9189    0.8078      9711
   is_Normal     0.9225    0.7305    0.8153     12833

    accuracy                         0.8116     22544
   macro avg     0.8215    0.8247    0.8115     22544
weighted avg     0.8355    0.8116    0.8121     22544

[[8923  788]
 [3459 9374]]
